In [1]:
import pandas as pd
import numpy as np

In [2]:
import pandas as pd

path = r"C:\Users\putri\SIAB_CBL\data\BBCA.JK_extended.csv"
df = pd.read_csv(path)

df.head()


,Date,Open,High,Low,Close,Volume
0,2014-12-08 00:00:00+00:00,2655.0,2680.0,2645.0,2655.0,37435500
1,2014-12-09 00:00:00+00:00,2660.0,2680.0,2630.0,2650.0,38254000
2,2014-12-10 00:00:00+00:00,2630.0,2675.0,2630.0,2660.0,32998000
3,2014-12-11 00:00:00+00:00,2660.0,2675.0,2645.0,2655.0,54459500
4,2014-12-12 00:00:00+00:00,2640.0,2665.0,2640.0,2650.0,50302000


In [3]:
import pandas as pd
import numpy as np

# pilih kolom yang dipakai
df = df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]

In [4]:
df['Date'] = pd.to_datetime(df['Date'], utc=True).dt.tz_localize(None)


In [5]:
df.isnull().sum()

Date      0
Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64

In [6]:
df.duplicated().sum()

np.int64(0)

In [7]:
df.dtypes

Date      datetime64[ns]
Open             float64
High             float64
Low              float64
Close            float64
Volume             int64
dtype: object

In [8]:
threshold = 0.003  # 0.3%

df['Return'] = (df['Close'].shift(-1) - df['Close']) / df['Close']

df['Label'] = np.where(df['Return'] > threshold, 1, 0)
df = df[:-1]


In [9]:
label_dist = df['Label'].value_counts().rename(index={1:'Naik', 0:'Turun'})
label_percent = (df['Label'].value_counts(normalize=True) * 100).rename(index={1:'Naik', 0:'Turun'})

pd.DataFrame({
    'Jumlah Data': label_dist,
    'Persentase (%)': label_percent.round(2)
})

,Jumlah Data,Persentase (%)
Label,,
Turun,1714,62.88
Naik,1012,37.12


In [10]:
# Moving Average 5 hari
df['MA5'] = df['Close'].rolling(window=5).mean()

# Moving Average 10 hari
df['MA10'] = df['Close'].rolling(window=10).mean()

df['Return_1'] = df['Close'].pct_change()
df['MA_diff'] = df['MA5'] - df['MA10']
df = df.dropna()


In [11]:
df = df.dropna().reset_index(drop=True)

In [12]:
df.head()

,Date,Open,High,Low,Close,Volume,Return,Label,MA5,MA10,Return_1,MA_diff
0,2014-12-19,2630.0,2650.0,2620.0,2635.0,83389500,-0.007590,0,2596.0,2625.0,0.015414,-29.0
1,2014-12-22,2650.0,2650.0,2615.0,2615.0,33301000,0.005736,1,2597.0,2621.0,-0.007590,-24.0
2,2014-12-23,2640.0,2640.0,2625.0,2630.0,21132000,0.003802,1,2609.0,2619.0,0.005736,-10.0
3,2014-12-24,2650.0,2655.0,2635.0,2640.0,19135500,-0.003788,0,2623.0,2617.0,0.003802,6.0
4,2014-12-29,2630.0,2650.0,2625.0,2630.0,24315000,-0.001901,0,2630.0,2614.5,-0.003788,15.5


In [13]:
X = df[
    [
        'Open',
        'High',
        'Low',
        'Close',
        'Volume',
        'MA5',
        'MA10',
        'Return',
        'Return_1',
        'MA_diff'
    ]
]

y = df['Label']


In [14]:
split_index = int(len(df) * 0.8)

X_train = X.iloc[:split_index]
X_test  = X.iloc[split_index:]

y_train = y.iloc[:split_index]
y_test  = y.iloc[split_index:]

In [15]:
print("Total data :", len(df))
print("Train data :", len(X_train))
print("Test data  :", len(X_test))


Total data : 2717
Train data : 2173
Test data  : 544


In [16]:
pip install xgboost


Note: you may need to restart the kernel to use updated packages.


In [17]:
from xgboost import XGBClassifier

model = XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    eval_metric='logloss'
)

model.fit(X_train, y_train)


,"objective objective: str | xgboost.sklearn._SklObjWProto | typing.Callable[[typing.Any, typing.Any], typing.Tuple[numpy.ndarray, numpy.ndarray]] | NoneSpecify the learning task and the corresponding learning objective or a customobjective function to be used.For custom objective, see :doc:`/tutorials/custom_metric_obj` and:ref:`custom-obj-metric` for more information, along with the end note forfunction signatures.",'binary:logistic'
,"base_score base_score: float | typing.List[float] | NoneThe initial prediction score of all instances, global bias.",None
,booster,None
,"callbacks callbacks: typing.List[xgboost.callback.TrainingCallback] | NoneList of callback functions that are applied at end of each iteration.It is possible to use predefined callbacks by using:ref:`Callback API `... note:: States in callback are not preserved during training, which means callback objects can not be reused for multiple training sessions without reinitialization or deepcopy... code-block:: python for params in parameters_grid: # be sure to (re)initialize the callbacks before each run callbacks = [xgb.callback.LearningRateScheduler(custom_rates)] reg = xgboost.XGBRegressor(**params, callbacks=callbacks) reg.fit(X, y)",None
,colsample_bylevel colsample_bylevel: float | NoneSubsample ratio of columns for each level.,None
,colsample_bynode colsample_bynode: float | NoneSubsample ratio of columns for each split.,None
,colsample_bytree colsample_bytree: float | NoneSubsample ratio of columns when constructing each tree.,0.8
,"device device: str | None.. versionadded:: 2.0.0Device ordinal, available options are `cpu`, `cuda`, and `gpu`.",None
,"early_stopping_rounds early_stopping_rounds: int | None.. versionadded:: 1.6.0- Activates early stopping. Validation metric needs to improve at least once in every **early_stopping_rounds** round(s) to continue training. Requires at least one item in **eval_set** in :py:meth:`fit`.- If early stopping occurs, the model will have two additional attributes: :py:attr:`best_score` and :py:attr:`best_iteration`. These are used by the :py:meth:`predict` and :py:meth:`apply` methods to determine the optimal number of trees during inference. If users want to access the full model (including trees built after early stopping), they can specify the `iteration_range` in these inference methods. In addition, other utilities like model plotting can also use the entire model.- If you prefer to discard the trees after `best_iteration`, consider using the callback function :py:class:`xgboost.callback.EarlyStopping`.- If there's more than one item in **eval_set**, the last entry will be used for early stopping. If there's more than one metric in **eval_metric**, the last metric will be used for early stopping.",None
,enable_categorical enable_categorical: boolSee the same parameter of :py:class:`DMatrix` for details.,False
,"eval_metric eval_metric: str | typing.List[str | typing.Callable] | typing.Callable | None.. versionadded:: 1.6.0Metric used for monitoring the training result and early stopping. It can be astring or list of strings as names of predefined metric in XGBoost (See:doc:`/parameter`), one of the metrics in :py:mod:`sklearn.metrics`, or anyother user defined metric that looks like `sklearn.metrics`.If custom objective is also provided, then custom metric should implement thecorresponding reverse link function.Unlike the `scoring` parameter commonly used in scikit-learn, when a callableobject is provided, it's assumed to be a cost function and by default XGBoostwill minimize the result during early stopping.For advanced usage on Early stopping like directly choosing to maximize insteadof minimize, see :py:obj:`xgboost.callback.EarlyStopping`.See :doc:`/tutorials/custom_metric_obj` and :ref:`custom-obj-metric` for moreinformation... code-block:: python from sklearn.datasets import load_diabetes from sklearn.metrics import mean_absolute_error X, y = load_diabetes(return_X_y=True) reg = xgb.XGBRegressor( tree_metho

In [18]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_pred = model.predict(X_test)

print("Akurasi:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=['Turun','Naik']))


Akurasi: 1.0
Confusion Matrix:
 [[351   0]
 [  0 193]]
              precision    recall  f1-score   support

       Turun       1.00      1.00      1.00       351
        Naik       1.00      1.00      1.00       193

    accuracy                           1.00       544
   macro avg       1.00      1.00      1.00       544
weighted avg       1.00      1.00      1.00       544



In [19]:
pip install yfinance


Note: you may need to restart the kernel to use updated packages.


In [20]:
import yfinance as yf
import pandas as pd
import numpy as np

ticker = yf.Ticker("BBCA.JK")

# ambil 3 bulan terakhir (aman untuk hitung MA)
df_live = ticker.history(period="3mo", interval="1d").reset_index()
df_live.head()


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2025-10-09 00:00:00+07:00,7326.567221,7649.432895,7252.059758,7500.417969,293567400,0.0,0.0
1,2025-10-10 00:00:00+07:00,7401.074473,7401.074473,7301.731191,7351.402832,107426000,0.0,0.0
2,2025-10-13 00:00:00+07:00,7252.059687,7326.567150,7252.059687,7276.895508,101847900,0.0,0.0
3,2025-10-14 00:00:00+07:00,7227.224005,7301.731469,7202.388184,7202.388184,151862300,0.0,0.0
4,2025-10-15 00:00:00+07:00,7202.388184,7301.731469,7177.552362,7202.388184,133206400,0.0,0.0


In [21]:
import pandas as pd

# ganti path sesuai file kamu
df_old = pd.read_csv("BBCA.JK_10_years.csv")

df_old['Date'] = pd.to_datetime(df_old['Date'])
df_old.tail()


FileNotFoundError: [Errno 2] No such file or directory: 'BBCA.JK_10_years.csv'

In [ ]:
import yfinance as yf

ticker = yf.Ticker("BBCA.JK")

df_new = ticker.history(
    start="2024-12-04",
    end=None,          # sampai tanggal terbaru
    interval="1d"
).reset_index()

df_new.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2024-12-04 00:00:00+07:00,9806.442656,9878.726017,9685.970387,9830.537109,97876300,0.0,0.0
1,2024-12-05 00:00:00+07:00,9878.725586,9975.103397,9854.631133,9878.725586,96904400,0.0,0.0
2,2024-12-06 00:00:00+07:00,9734.159885,9806.443250,9685.970975,9710.065430,68247200,0.0,0.0
3,2024-12-09 00:00:00+07:00,9661.875627,9975.103516,9637.781175,9975.103516,64458400,0.0,0.0
4,2024-12-10 00:00:00+07:00,9951.009063,9975.103516,9902.820157,9975.103516,61230400,0.0,0.0


In [ ]:
df_new = df_new.rename(columns={
    'Open': 'Open',
    'High': 'High',
    'Low': 'Low',
    'Close': 'Close',
    'Volume': 'Volume'
})

df_new = df_new[['Date','Open','High','Low','Close','Volume']]

In [ ]:
df_all = pd.concat([df_old[['Date','Open','High','Low','Close','Volume']], df_new])

# hapus duplikat tanggal (jaga-jaga)
df_all = df_all.drop_duplicates(subset='Date')

# urutkan waktu
df_all = df_all.sort_values('Date').reset_index(drop=True)

df_all.tail()

,Date,Open,High,Low,Close,Volume
2722,2025-12-23 00:00:00+07:00,8100.0,8125.0,7975.0,8025.0,85044100
2723,2025-12-24 00:00:00+07:00,8025.0,8075.0,8000.0,8025.0,71260500
2724,2025-12-29 00:00:00+07:00,8025.0,8025.0,7950.0,8025.0,78399400
2725,2025-12-30 00:00:00+07:00,7950.0,8175.0,7950.0,8075.0,101995600
2726,2026-01-02 00:00:00+07:00,8075.0,8075.0,8000.0,8025.0,32612800


In [22]:
df_all.to_csv("BBCA.JK_extended.csv", index=False)


NameError: name 'df_all' is not defined

In [ ]:
import joblib

joblib.dump(model, "siab_model.pkl")

['siab_model.pkl']

In [ ]:
def predict_signal(input_data):
    input_scaled = scaler.transform(input_data)
    return model.predict(input_scaled)